In [ ]:
#r "nuget: Elastic.Clients.Elasticsearch, 8.15.10"
#r "nuget: System.Net.Http.Json, 8.0.1"
#!import ./Utils.cs
#!import ../_infra/get-connection-string.ipynb

## Initialize the Elasticsearch client

We can instantiate the [Elastic.Clients.Elasticsearch](https://github.com/elastic/elasticsearch-net) client.

Make sure you have an instance of Elasticsearch running, see [setup-elastic-infrastructure.ipynb](../_infra/setup-elastic-infrastructure.ipynb) for more details.


In [ ]:
using Elastic.Transport;
using Elastic.Clients.Elasticsearch;
using Elastic.Transport.Products.Elasticsearch;

var elasticSettings = new ElasticsearchClientSettings(connectionString)
    .DisableDirectStreaming()
    .ServerCertificateValidationCallback(CertificateValidations.AllowAll);

var client = new ElasticsearchClient(elasticSettings);

## Test the Client

Before you continue, confirm that the client has connected with this test.


In [ ]:
var info = await client.InfoAsync();

display(info);

## Setup the Embedding Model


In [ ]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.0.0-preview.*"
#r "nuget: Azure.AI.OpenAI, 2.0.0"
#r "nuget: dotenv.net, 3.2.1"

In [ ]:
using Azure.AI.OpenAI;
using Microsoft.Extensions.AI;

IEmbeddingGenerator<string,Embedding<float>> generator =
    new AzureOpenAIClient(
        new Uri(envs["AZURE_OPENAI_ENDPOINT"]),
        new System.ClientModel.ApiKeyCredential(envs["AZURE_OPENAI_APIKEY"]))
            .AsEmbeddingGenerator(modelId: "text-embedding-3-small");

## Index some test data
Our client is set up and connected to our Elastic deployment. Now we need some data to test out the basics of Elasticsearch queries. We'll use a small index of books with the following fields

In [ ]:
using System.Text.Json.Serialization;

public class Book
{
    [JsonPropertyName("title")]
    public string Title { get; set; }

    [JsonPropertyName("summary")]
    public string Summary { get; set; }

    [JsonPropertyName("publish_date")]
    public DateTime PublishDate { get; set; }

    [JsonPropertyName("num_reviews")]
    public int NumReviews { get; set; }

    [JsonPropertyName("publisher")]
    public string Publisher { get; set; }


    public float[] TitleVector { get; set; }
}

In [ ]:
using Elastic.Clients.Elasticsearch;
using Elastic.Clients.Elasticsearch.IndexManagement;
using Elastic.Clients.Elasticsearch.Mapping;

var textEmeddingDimension = 384;

const string BookIndex = "book_index";
var indexDescriptor = new CreateIndexRequestDescriptor<Book>(BookIndex)
    .Mappings(m => m
        .Properties(pp => pp
            .Text(p => p.Title)
            .DenseVector(
                Infer.Property<Book>(p => p.TitleVector),
                d => d.Dims(textEmeddingDimension).Index(true).Similarity(DenseVectorSimilarity.Cosine))
            .Text(p => p.Summary)
            .Date(p => p.PublishDate)
            .IntegerNumber(p => p.NumReviews)
            .Keyword(p => p.Publisher)
        )
    );

In [ ]:
var indexResponse = await client.Indices.CreateAsync<Book>(indexDescriptor);

// display(indexResponse);
ToJson(DumpGetRequest(indexResponse.DebugInformation))
    .DisplayAs("application/json");

In [ ]:
using System.Net.Http;
using System.Net.Http.Json;

var http = new HttpClient();
var url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json";
var books =  await http.GetFromJsonAsync<Book[]>(url);

display(books);


In [ ]:
async Task<float[]> ToEmbedding(string text) {
    GeneratedEmbeddings<Embedding<float>> embeddings = await generator
        .GenerateAsync(text, new EmbeddingGenerationOptions{
            AdditionalProperties = new AdditionalPropertiesDictionary{
                {"dimensions", textEmeddingDimension}
            }
        });

    return embeddings.First().Vector.ToArray();
}

var embedding = await ToEmbedding("The quick brown fox jumps over the lazy dog");
embedding.Length


In [ ]:
foreach(var book in books)
{
    book.TitleVector = await ToEmbedding(book.Title);
    display(book.Title);
}

In [ ]:
var bulkResponse = await client.BulkAsync(BookIndex, d => d
    .IndexMany<Book>(books, (bd, b) => bd.Index(BookIndex)));

bulkResponse.Display();
// display(DumpGetRequest(bulkResponse));

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Index(BookIndex)
    .Query(q => q.Match(m => m.Field(f => f.Title).Query("JavaScript")))
);

ToJson(DumpGetRequest(searchResponse)).Display();

searchResponse.Display();

In [ ]:

var searchQuery = "JavaScript";
var queryEmbedding = await ToEmbedding(searchQuery);
queryEmbedding.Length.Display();
var searchResponse = await client.SearchAsync<Book>(s => s
    .Index(BookIndex)
    .Knn(d => d
        .Field(f => f.TitleVector)
        .QueryVector(queryEmbedding)
        .k(10)
        .NumCandidates(100))
);

ToJson(DumpGetRequest(searchResponse)).Display();

searchResponse.Display();

In [ ]:
await client.Indices.DeleteAsync(BookIndex);